In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import numpy as np 
import pandas as pd 
import bz2
import gc
import chardet
import re
import os
import random

# Pre-processing

In [3]:
#Checking files in Kaggle
# List data files that are connected to the kernel
os.listdir('../input')

['test.ft.txt.bz2', 'train.ft.txt.bz2']

In [4]:
# Read Train & Test Files

#Kaggle
train_file = bz2.BZ2File('../input/train.ft.txt.bz2')
test_file = bz2.BZ2File('../input/test.ft.txt.bz2')

#Localhost
#train_file = bz2.BZ2File('C:/Users/Lenovo/Documents/GitHub/Datasets/amazonreviews/train.ft.txt.bz2')
#test_file = bz2.BZ2File('C:/Users/Lenovo/Documents/GitHub/Datasets/amazonreviews/test.ft.txt.bz2')

#Localhost - Versión recortada del archivo
#train_file = bz2.BZ2File('C:/Users/Lenovo/Documents/GitHub/Datasets/amazonreviews/Version_Recortada/r_train.ft.txt.bz2')
#test_file = bz2.BZ2File('C:/Users/Lenovo/Documents/GitHub/Datasets/amazonreviews/Version_Recortada/r_test.ft.txt.bz2')

#Create Lists containing Train & Test sentences
train_file_lines = train_file.readlines()
test_file_lines = test_file.readlines()

#Convert from raw binary strings to strings that can be parsed
train_file_lines = [x.decode('utf-8') for x in train_file_lines]
test_file_lines = [x.decode('utf-8') for x in test_file_lines]

In [5]:
#Delete memory reference (?)
del train_file, test_file
#Garbage collector
gc.collect()

0

In [6]:
print("Cantidad de elementos del Training Set: {}".format(len(train_file_lines)))
print("Cantidad de elementos del Testing Set: {}".format(len(test_file_lines)))

Cantidad de elementos del Training Set: 3600000
Cantidad de elementos del Testing Set: 400000


## Clean data

In [7]:
# Change labels: __label__1 -> 0 (Negative) / __label__2 -> 1 (Positive)
train_labels = [0 if x.split(' ')[0] == '__label__1' else 1 for x in train_file_lines]
test_labels = [0 if x.split(' ')[0] == '__label__1' else 1 for x in test_file_lines]

# Make everything Lower Case
train_sentences = [x.split(' ', 1)[1][:-1].lower() for x in train_file_lines]

for i in range(len(train_sentences)):
    train_sentences[i] = re.sub('\d','0',train_sentences[i])
    
test_sentences = [x.split(' ', 1)[1][:-1].lower() for x in test_file_lines]

for i in range(len(test_sentences)):
    test_sentences[i] = re.sub('\d','0',test_sentences[i])

# Modify URLs to <url>
for i in range(len(train_sentences)):
    if 'www.' in train_sentences[i] or 'http:' in train_sentences[i] or 'https:' in train_sentences[i] or '.com' in train_sentences[i]:
        train_sentences[i] = re.sub(r"([^ ]+(?<=\.[a-z]{3}))", "<url>", train_sentences[i])
        
for i in range(len(test_sentences)):
    if 'www.' in test_sentences[i] or 'http:' in test_sentences[i] or 'https:' in test_sentences[i] or '.com' in test_sentences[i]:
        test_sentences[i] = re.sub(r"([^ ]+(?<=\.[a-z]{3}))", "<url>", test_sentences[i])

## Checking data before and after cleaning

In [8]:
#Random
r = random.randint(1,len(train_file_lines))

#Before
print("Data before cleaning:\n{}".format(train_file_lines[r-1:r]))

#After
print("\nData after cleaning:\n{}".format((train_sentences[r-1:r])))

#Labels
print("\nLabel:{}".format(train_labels[r-1:r]))

Data before cleaning:
['__label__2 great book: This book has almost all of the information you need to build you own barn and fencing. I would recomend it!\n']

Data after cleaning:
['great book: this book has almost all of the information you need to build you own barn and fencing. i would recomend it!']

Label:[1]


### Output
From the above output it can be seen that each sentence begins with it's sentiment (label1 -> Negative, label2 -> Positive), which is then followed by the review and ends with a newline character \n.

So, first I go convert all the labels to O(Negative) and 1(Positive) and store it in lists that only contain the label values. After this, I store the remainder of the sentence excluding the newline character in lowercase in lists. Also, convert all numbers to 0.


In [9]:
#Delete memory reference (?)
del train_file_lines, test_file_lines
#Garbage collector
gc.collect()

0

## Text Pre-processing

In [10]:
from keras.preprocessing import text, sequence

#Base definitions for text preprocessing
max_features = 20000
maxlen = 100

/opt/conda/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [11]:
#Tokenizer definition
#Filtro caracteres especiales usando el Tokenizer de keras.
tokenizer = text.Tokenizer(num_words=max_features, filters='!"#$%&()*+,-./:;<=>?@[\]^_`{|}~')

#Fit on text -> Only the train dataset !!!
tokenizer.fit_on_texts(train_sentences)

#Training set
tokenized_train = tokenizer.texts_to_sequences(train_sentences)
X_train = sequence.pad_sequences(tokenized_train, maxlen=maxlen)

#Test set
tokenized_test = tokenizer.texts_to_sequences(test_sentences)
X_test = sequence.pad_sequences(tokenized_test, maxlen=maxlen)

In [12]:
#Print a random matrix
X_train[r]
# summarize what was learned -> Si quiero ver el tokenizer que aprendio usando los 2 parametros (Max_features,max_length)
#print(t.word_counts)
#print(t.document_count)
#print(t.word_index)
#print(t.word_docs)

array([    0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,  1263,    37,     1,  2003,
        4185,     3,    93,    10,  9852,     2,    20,   312,     1,
        6435,   112,   206,     6,     9,   472,     4, 12491,  2110,
        3579,    16,     1, 11026,   273,   885,  1180,    10,     8,
         698,     9,  3708,   695,    10,  1420,  3815,   202,   523,
         970,    35,  2862,  3309,     2,    11,    43,     1,   915,
         450], dtype=int32)

### Validation dataset

In [13]:
from sklearn.model_selection import train_test_split
# Create a validation dataset
validation_size = 0.2
X_train, X_valid, train_labels, valid_labes = train_test_split(X_train, train_labels, test_size = validation_size)

In [14]:
#Delete memory reference (?)
del tokenized_test, tokenized_train, tokenizer, train_sentences, test_sentences
#Garbage collector
gc.collect()

0

## Model

In [15]:
from keras.utils import to_categorical
from keras.callbacks import ModelCheckpoint 
from keras.layers.core import Dense, Dropout, Activation, Flatten
from keras.layers.normalization import BatchNormalization
from keras import optimizers
from keras import initializers
from keras.models import Model, Sequential
from keras.layers import Convolution1D, MaxPooling1D, GlobalAveragePooling1D, BatchNormalization, LSTM, GRU, CuDNNGRU, CuDNNLSTM, concatenate, Input
from keras.layers.embeddings import Embedding

In [16]:
#Defino los parametros del modelo:
p = 0.10 #Dropout
lr = 0.0001 #Learning Rate
batch_size = 2048
epochs = 4 #Bajamos de 10 -> 4.

#Embedding size -> Ver para que sirve, todavia falta entenderlo?
embed_size = 128
#CNN_Filters
CNN_Filters = embed_size * 2
#RNN
RNN_Neurons = 128
time_steps = 0

In [17]:
# Creo el modelo
def model():
    #input
    inp = Input(shape=(maxlen, ))
    #Embedding
    x = Embedding(max_features, embed_size)(inp)
    x = Dropout(0.25)(x)
    #CNN
    x = Convolution1D(2*embed_size, kernel_size = 3)(x)
    x = Convolution1D(2*embed_size, kernel_size = 3)(x)
    #RNN
    x_a = GRU(512)(x)  
    x_b = GRU(512)(x)  
    #Concatenate
    x = concatenate([x_a, x_b])
    x = Dropout(0.5)(x)
    x = Dense(64, activation="relu")(x)
    x = Dropout(0.1)(x)
    x = Dense(1, activation="sigmoid")(x)
    #Input - Output
    model = Model(inputs=inp, outputs=x)
    #Optimizer
    ADAM = optimizers.Adam(lr=lr)
    model.compile(loss = 'binary_crossentropy', optimizer=ADAM, metrics=['binary_accuracy'])

    return model

model = model()
model.summary()

#Optimizers

#Accuracy en metrics es más generico, y depende de la LOSS.
#model.compile(loss = 'binary_crossentropy', optimizer=ADAM, metrics=['accuracy'])

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 100)          0                                            
__________________________________________________________________________________________________
embedding_1 (Embedding)         (None, 100, 128)     2560000     input_1[0][0]                    
__________________________________________________________________________________________________
dropout_1 (Dropout)             (None, 100, 128)     0           embedding_1[0][0]                
__________________________________________________________________________________________________
conv1d_1 (Conv1D)               (None, 98, 256)      98560       dropout_1[0][0]                  
__________________________________________________________________________________________________
conv1d_2 (

In [18]:
# Callbacks

## Callback para guardar pesos
checkpointer = ModelCheckpoint(filepath='Sentiment_Analysis_Amazon_Reviews.hdf5', monitor='val_loss'
                                   ,verbose=1, save_best_only=True, mode='min')
callbacks = [checkpointer]

In [19]:
# Fit del modelo -> Usando todo el dataset
"""
model.fit(X_train,train_labels
          ,epochs=epochs
          ,batch_size = batch_size          
          ,shuffle = True
          ,validation_data = (X_valid,valid_labes)
          ,callbacks=callbacks)
"""

# Fit del modelo -> Usando solo un fragmento del datasset
model.fit(X_train[:100000], train_labels[:100000]
          ,epochs=epochs
          ,batch_size = batch_size          
          ,shuffle = True
          ,validation_split=0.20
          ,callbacks=callbacks)

Train on 80000 samples, validate on 20000 samples
Epoch 1/4
80000/80000 [==============================] - 140s 2ms/step - loss: 0.6852 - binary_accuracy: 0.5705 - val_loss: 0.6510 - val_binary_accuracy: 0.6387

Epoch 00001: val_loss improved from inf to 0.65097, saving model to Sentiment_Analysis_Amazon_Reviews.hdf5
Epoch 2/4
80000/80000 [==============================] - 132s 2ms/step - loss: 0.5290 - binary_accuracy: 0.7341 - val_loss: 0.3992 - val_binary_accuracy: 0.8212

Epoch 00002: val_loss improved from 0.65097 to 0.39918, saving model to Sentiment_Analysis_Amazon_Reviews.hdf5
Epoch 3/4
80000/80000 [==============================] - 132s 2ms/step - loss: 0.3700 - binary_accuracy: 0.8389 - val_loss: 0.3550 - val_binary_accuracy: 0.8456

Epoch 00003: val_loss improved from 0.39918 to 0.35495, saving model to Sentiment_Analysis_Amazon_Reviews.hdf5
Epoch 4/4
80000/80000 [==============================] - 133s 2ms/step - loss: 0.3179 - binary_accuracy: 0.8665 - val_loss: 0.3081 - va

## Test 

In [20]:
score, acc = model.evaluate(X_test, test_labels, batch_size=batch_size)
print('Test score:', score)
print('Test accuracy:', acc)

400000/400000 [==============================] - 171s 428us/step
Test score: 0.30853510454177857
Test accuracy: 0.8726525
